<a href="https://colab.research.google.com/github/ryannhally/Comps_Project_and_Essay/blob/main/CS_390_Train_modern_%26_classical_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate sacrebleu elotl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavi

In [ ]:
from datasets import load_dataset
from datasets import Dataset

In [ ]:
# Load dataset and filter null
axolotl_raw = load_dataset('somosnlp-hackathon-2022/Axolotl-Spanish-Nahuatl')
axolotl_filter = axolotl_raw.filter(lambda x:x['sp'] is not None)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.77k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/6.45M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20028 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20028 [00:00<?, ? examples/s]

In [ ]:
# Check
print(axolotl_filter.num_rows)

{'train': 20024}


In [ ]:
# Normalize
import elotl.corpus
import elotl.nahuatl.orthography
n = elotl.nahuatl.orthography.Normalizer("sep")

translation = []
idx = []

i = 0
for example in axolotl_filter['train']:
  idx.append(i)
  translation.append({'es': axolotl_filter['train'][i]['sp'], 'na': n.normalize(axolotl_filter['train'][i]['nah'])})
  i += 1

In [ ]:
print(len(translation))
print(len(idx))

20024
20024


In [ ]:
# Make dictionary
data = { "id": idx, "translation": translation}

books = Dataset.from_dict(data)
books = books.train_test_split(test_size=0.1)

In [ ]:
print(books)
print(books['train'][0])
print(books['train'][1818] )
print(books['train'][10000])
print(books['train'][500])

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 18021
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 2003
    })
})
{'id': 11199, 'translation': {'es': 'Es árbol (kuouit).', 'na': 'in kuouit.'}}
{'id': 12104, 'translation': {'es': 'CAFETO', 'na': 'kajfenkuouit'}}
{'id': 8038, 'translation': {'es': 'Y mientras estuvieron allí, tampoco se atrevían a hacer sus necesidades corporales junto al Popocatépetl, sino que iban a hacerlas un poco más lejos, cerca de Atlixco, y por eso ahora el lugar se llama Cuitlaóztoc; éste fue el segundo sitio donde los xochtecas olmecas quiyahuiztecas cocolcas dejaron en esos nombres su marca y su recuerdo.', 'na': 'au inik ompa katka anouel onkan maxixaya in itlan popokatepetl, okno nepa in onmaxixaya ye okachi ueka inauak atlixko, ik axkan motokayotia kuitlaostok; ikokkan kitlalike inneska intoka imelnamikoka in xochteka in ulmeka in kiyauisteka in kukolka.'}}
{'id': 3227, '

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

tokenizer = AutoTokenizer.from_pretrained('google-t5/t5-small')
model = AutoModelForSeq2SeqLM.from_pretrained('google-t5/t5-small')

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Pre-processing function
source_lang = "es"

target_lang = "na"

prefix = "translate Spanish to Nahuatl: "


def preprocess_function(examples):

    inputs = [prefix + example[source_lang] for example in examples["translation"]]

    targets = [example[target_lang] for example in examples["translation"]]

    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    with tokenizer.as_target_tokenizer():

        labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [ ]:
# Tokenize
tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/18021 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2003 [00:00<?, ? examples/s]

In [ ]:
# Check
print(tokenized_books)
print(tokenized_books['train'][0])
print(tokenized_books['train'][1])
print(tokenized_books['train'][200])
print(tokenized_books['train'][10000])

DatasetDict({
    train: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 18021
    })
    test: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2003
    })
})
{'id': 11199, 'translation': {'es': 'Es árbol (kuouit).', 'na': 'in kuouit.'}, 'input_ids': [13959, 5093, 12, 22885, 76, 144, 40, 10, 1122, 3, 2975, 52, 4243, 41, 2729, 1063, 155, 137, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [16, 3, 2729, 1063, 155, 5, 1]}
{'id': 2954, 'translation': {'es': 'Florea en marzo y da fruto en abril y mayo.', 'na': 'xochiyoua marso uan taki abril uan mayo.'}, 'input_ids': [13959, 5093, 12, 22885, 76, 144, 40, 10, 7542, 15, 9, 3, 35, 3157, 172, 32, 3, 63, 836, 2515, 76, 235, 3, 35, 3, 17639, 40, 3, 63, 164, 32, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
# Data collator
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
# Sign into HuggingFace
from google.colab import userdata
userdata.get('HuggingFaceKey')

from huggingface_hub import notebook_login

notebook_login('HuggingFaceKey')

In [ ]:
# Hyperparemeters
training_args = Seq2SeqTrainingArguments(

    output_dir="ryannhally/t5-small-nah-all",

    evaluation_strategy="steps",

    eval_steps=1000,

    save_steps=5000,

    learning_rate=2e-5,

    per_device_train_batch_size=16,

    per_device_eval_batch_size=16,

    weight_decay=0.01,

    num_train_epochs=280,

    save_total_limit=3,

    fp16=True,

    push_to_hub=True

)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Seq2SeqTrainer(

    model=model,

    args=training_args,

    train_dataset=tokenized_books["train"],

    eval_dataset=tokenized_books["test"],

    tokenizer=tokenizer,

    data_collator=data_collator,

)

<ipython-input-16-7c93e7e4f211>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
1000,3.499600,3.112193
2000,3.155900,2.834970
3000,2.986100,2.682931
4000,2.853500,2.576893
5000,2.754800,2.495681
6000,2.683200,2.429733
7000,2.617900,2.376122
8000,2.575400,2.328909
9000,2.525600,2.287040
10000,2.485100,2.248929
